In [3]:
from rag.vector_base import VectorStore
from rag.utils import ReadFiles
from rag.embeddings import BgeEmbedding
from rag.llm import GeminiChat

## 构建向量数据库

In [19]:
reader = ReadFiles('../data/1')
docs = reader.get_content(max_token_len=1024, cover_content=256)
len(docs)

1682

In [20]:
embedding = BgeEmbedding(device='mps')
vector_store = VectorStore(docs)
vectors = vector_store.get_vector(embedding)
vector_store.persist(file='d2l')

Calculating embeddings: 100%|██████████| 1682/1682 [24:24<00:00,  1.15it/s]


## 加载向量数据库并测试

In [29]:
vector_store = VectorStore()
vector_store.load_vector(file='d2l')
embedding = BgeEmbedding(device='mps')
question = '缩放点积注意力的原理？'
content = vector_store.query(question, EmbeddingModel=embedding, k=1)[0]
content

'是独立的随机变量，\n并且都满足零均值和单位方差，\n那么两个向量的点积的均值为$0$，方差为$d$。\n为确保无论向量长度如何，\n点积的方差在不考虑向量长度的情况下仍然是$1$，\n我们再将点积除以$\\sqrt{d}$，\n则缩放点积注意力（scaled dot-product attention）评分函数为：\n$$a(\\mathbf q, \\mathbf k) = \\mathbf{q}^\\top \\mathbf{k}  /\\sqrt{d}.$$\n在实践中，我们通常从小批量的角度来考虑提高效率，\n例如基于$n$个查询和$m$个键－值对计算注意力，\n其中查询和键的长度为$d$，值的长度为$v$。\n查询$\\mathbf Q\\in\\mathbb R^{n\\times d}$、\n键$\\mathbf K\\in\\mathbb R^{m\\times d}$和\n值$\\mathbf V\\in\\mathbb R^{m\\times v}$的缩放点积注意力是：\n$$ \\mathrm{softmax}\\left(\\frac{\\mathbf Q \\mathbf K^\\top }{\\sqrt{d}}\\right) \\mathbf V \\in \\mathbb{R}^{n\\times v}.$$\n:eqlabel:eq_softmax_QK_V\n下面的缩放点积注意力的实现使用了暂退法进行模型正则化。\n```{.python .input}\n@save\nclass DotProductAttention(nn.Block):\n    """缩放点积注意力"""\n    def init(self, dropout, kwargs):\n        super(DotProductAttention, self).init(kwargs)\n        self.dropout = nn.Dropout(dropout)\n# queries的形状：(batch_size，查询的个数，d)\n# keys的形状：(batch_size，“键－值”对的个数，d)\n# values的形状：(batch_size，“键－值”对的个数，值的维度)\n# valid_lens的形状:(batch_size，

In [30]:
chat = GeminiChat(model='gemini-2.5-flash')
res = chat.chat(question, [], content)
from IPython.display import Markdown, display
display(Markdown(res))

缩放点积注意力的原理如下：

为了确保点积的方差在不考虑向量长度的情况下仍然是$1$，无论向量长度如何，我们会将查询向量$\mathbf q$和键向量$\mathbf k$的点积$\mathbf{q}^\top \mathbf{k}$除以$\sqrt{d}$，其中$d$是向量的维度（长度）。

因此，缩放点积注意力（scaled dot-product attention）的评分函数为：
$$a(\mathbf q, \mathbf k) = \mathbf{q}^\top \mathbf{k} /\sqrt{d}.$$

在实际应用中，当处理批量数据时，对于查询矩阵$\mathbf Q$、键矩阵$\mathbf K$和值矩阵$\mathbf V$，缩放点积注意力计算为：
$$ \mathrm{softmax}\left(\frac{\mathbf Q \mathbf K^\top }{\sqrt{d}}\right) \mathbf V $$
这里，$\mathbf Q \mathbf K^\top$计算了所有查询和键之间的点积，然后通过除以$\sqrt{d}$进行缩放，接着应用softmax函数得到注意力权重，最后将这些权重与值矩阵$\mathbf V$相乘。

## 召回的RAG

In [25]:
from rag.reranker import BgeReranker

reranker = BgeReranker()
vector_store = VectorStore()
vector_store.load_vector(file='stochastic_process')

question = '残差连接是什么？'
# 从向量数据库中查询出最相似的3个文档
content = vector_store.query(question, EmbeddingModel=embedding, k=3)
# 从一阶段查询结果中用Reranker再次筛选出最相似的2个文档
rerank_content = reranker.rerank(question, content, k=2)
# 最后选择最相似的文档, 交给LLM作为可参考上下文
best_content = rerank_content[0]

chat = GeminiChat(model='gemini-2.5-flash')
res = chat.chat(question, [], best_content)
from IPython.display import Markdown, display
display(Markdown(res))


残差连接是残差网络（ResNet）中残差块的核心组成部分。它的主要思想是，不是直接学习一个理想映射 $f(\mathbf{x})$，而是学习一个残差映射 $f(\mathbf{x}) - \mathbf{x}$。

在残差块中，原始输入 $\mathbf{x}$ 会被“连接”到（通常是通过跳跃连接或快捷连接）经过一些层处理后的输出上。具体来说，如果中间层学习到了残差映射 $f(\mathbf{x}) - \mathbf{x}$，那么最终的输出就是这个残差映射加上原始输入 $\mathbf{x}$，即 $(f(\mathbf{x}) - \mathbf{x}) + \mathbf{x} = f(\mathbf{x})$。

这种设计使得网络更容易优化。尤其当理想映射 $f(\mathbf{x})$ 接近于恒等映射 $f(\mathbf{x}) = \mathbf{x}$ 时，残差映射 $f(\mathbf{x}) - \mathbf{x}$ 就会接近于零，网络只需要学习微小的波动。这有助于解决深度神经网络中层数增加时难以训练的问题，并确保新模型至少可以达到原有模型的性能（通过将新增层的权重和偏置设为0，使其学习恒等映射）。

In [26]:
embedding._model

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(8194, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elem

In [28]:
reranker._model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=